<h1>LSTM with word2vec embeddings</h1><br>
https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings/

In [1]:
!ls

initial_kaggle_download_zip_files    sample_submission.csv  train.csv
LSTM with word2vec embeddings.ipynb  test.csv


In [2]:
!mkdir input

Jesus! This site: https://github.com/andrewdblevins/beyond_word2vec tells me that I'm about to download a 3Gb+ file 'GoogleNews-vectors-negative300.bin'<br>
<h3>Download the pretrained google news word vectors</h3><br>
<b><i>warning: This file is 3.6GB</i></b>

In [3]:
!wget https://s3.amazonaws.com/mordecai-geo/GoogleNews-vectors-negative300.bin.gz

--2017-11-30 11:06:11--  https://s3.amazonaws.com/mordecai-geo/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.18.251
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.18.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/octet-stream]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  20.2MB/s    in 89s     

2017-11-30 11:07:41 (17.6 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [4]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev             30G     0   30G   0% /dev
tmpfs           6.0G  8.8M  6.0G   1% /run
/dev/xvda1      126G   13G  108G  11% /
tmpfs            30G     0   30G   0% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs            30G     0   30G   0% /sys/fs/cgroup
tmpfs           6.0G   12K  6.0G   1% /run/user/1000


In [5]:
# Unzip the gz file
!gunzip GoogleNews-vectors-negative300.bin.gz

In [6]:
ls

GoogleNews-vectors-negative300.bin   sample_submission.csv
initial_kaggle_download_zip_files/   test.csv
input/                               train.csv
LSTM with word2vec embeddings.ipynb


In [7]:
# Next step - move the train and test csv files into the "input" folder
!mv *.csv input/

In [8]:
ls

GoogleNews-vectors-negative300.bin  input/
initial_kaggle_download_zip_files/  LSTM with word2vec embeddings.ipynb


In [9]:
# Move the bin file into the input folder too
mv *.bin input/

In [1]:
!ls

initial_kaggle_download_zip_files  LSTM with word2vec embeddings-backup01.ipynb
input				   LSTM with word2vec embeddings.ipynb


In [2]:
ls input

GoogleNews-vectors-negative300.bin  sample_submission.csv  test.csv  train.csv


<H1> Restarting the script in chunks from the start</H1>
Because the above was basically setting it up....

In [4]:
'''
Single model may achieve LB scores at around 0.29+ ~ 0.30+
Average ensembles can easily get 0.28+ or less
Don't need to be an expert of feature engineering
All you need is a GPU!!!!!!!

The code is tested on Keras 2.0.0 using Tensorflow backend, and Python 2.7

According to experiments by kagglers, Theano backend with GPU may give bad LB scores while
        the val_loss seems to be fine, so try Tensorflow backend first please
'''

########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [5]:
########################################
## set directories and parameters
########################################
BASE_DIR = 'input/'
EMBEDDING_FILE = BASE_DIR + 'GoogleNews-vectors-negative300.bin'
                            #GoogleNews-vectors-negative300.bin
TRAIN_DATA_FILE = BASE_DIR + 'train.csv'
TEST_DATA_FILE = BASE_DIR + 'test.csv'
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

num_lstm = np.random.randint(175, 275)
num_dense = np.random.randint(100, 150)
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25

act = 'relu'
re_weight = True # whether to re-weight classes to fit the 17.5% share in test set

STAMP = 'lstm_%d_%d_%.2f_%.2f'%(num_lstm, num_dense, rate_drop_lstm, \
        rate_drop_dense)

In [8]:
########################################
## index word vectors
########################################
print('Indexing word vectors')

In [ ]:
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, \
        binary=True)

In [ ]:
print('Found %s word vectors of word2vec' % len(word2vec.vocab))